In [11]:
# Standard libraries
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)

# Data manipulation and analysis
import pandas as pd
import numpy as np

from GEMS_TCO import kernels
from GEMS_TCO import data_preprocess 
from GEMS_TCO import kernels_new, kernels_reparam_space_time as kernels_reparam_space_time
from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import load_data
from GEMS_TCO import alg_optimization, alg_opt_Encoder
from GEMS_TCO import configuration as config

from typing import Optional, List, Tuple
from pathlib import Path
from json import JSONEncoder

from GEMS_TCO.data_loader import load_data2
import torch
import torch.optim as optim
import time

Load monthly data

In [2]:
space: List[str] = ['4', '4']
lat_lon_resolution = [int(s) for s in space]
mm_cond_number: int = 20
years = ['2024']
month_range = [7] 

output_path = input_path = Path(config.mac_estimates_day_path)
data_load_instance = load_data2(config.mac_data_load_path)


df_map, ord_mm, nns_map = data_load_instance.load_maxmin_ordered_data_bymonthyear(
lat_lon_resolution=lat_lon_resolution, 
mm_cond_number=mm_cond_number,
years_=years, 
months_=month_range,
lat_range=[0.0, 5.0],      
lon_range=[123.0, 133.0] 
)

#days: List[str] = ['0', '31']
#days_s_e = [int(d) for d in days]
#days_list = list(range(days_s_e[0], days_s_e[1]))

Subsetting data to lat: [0.0, 5.0], lon: [123.0, 133.0]


Load daily data applying max-min ordering

In [5]:
daily_aggregated_tensors = [] 
daily_hourly_maps = []        

for day_index in range(31):
  
    hour_start_index = day_index * 8
    hour_end_index = (day_index + 1) * 8
    #hour_end_index = day_index*8 + 1
    hour_indices = [hour_start_index, hour_end_index]
    
    # Load the data for the current day
    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
        df_map, 
        hour_indices, 
        ord_mm=ord_mm,  
        dtype=torch.float 
    )
    # Append the day's data to their respective lists
    daily_aggregated_tensors.append(day_aggregated_tensor)
    daily_hourly_maps.append(day_hourly_map) 

print(daily_aggregated_tensors[0].shape)
#print(daily_hourly_maps[0])

torch.Size([8960, 4])


Hyper parameters

In [7]:
v = 0.5 # smooth
mm_cond_number = 8
nheads = 300
#nheads = 1230
#lr = 0.01
#step = 80
#gamma_par = 0.5

# --- Placeholder Global Variables ---
# 💥 REVISED: Added lr, patience, factor. Removed step, gamma_par
lr=0.1
patience = 5       # Scheduler: Epochs to wait for improvement
factor = 0.5         # Scheduler: Factor to reduce LR by (e.g., 0.5 = 50% cut)
epochs=200

Admas full batch

In [12]:
import torch
import numpy as np
import time

# --- Assume global variables are set: ---
# daily_hourly_maps, daily_aggregated_tensors, nns_map
# lat_lon_resolution, v, mm_cond_number, nheads
# lr, patience, factor, epochs

# --- 2. Run optimization loop over pre-loaded data ---
day_indices = [0] 
for day_idx in day_indices:  

    daily_hourly_map = daily_hourly_maps[day_idx]
    daily_aggregated_tensor = daily_aggregated_tensors[day_idx]

    # --- 💥 Correct Parameter Initialization (SPATIO-TEMPORAL) ---
    init_sigmasq   = 15.0
    init_range_lat = 0.66 
    init_range_lon = 0.7 
    init_nugget    = 1.5
    init_beta      = 0.1  # <-- NEW (Temporal range)
    init_advec_lat = 0.02  # <-- NEW (Start with no advection)
    init_advec_lon = -0.08  # <-- NEW (Start with no advection)
    
    # Map model parameters to the 'phi' reparameterization
    init_phi2 = 1.0 / init_range_lon                # 1/range_lon
    init_phi1 = init_sigmasq * init_phi2            # sigmasq / range_lon
    init_phi3 = (init_range_lon / init_range_lat)**2  # (range_lon / range_lat)^2
    init_phi4 = init_beta**2                        # beta^2
    
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    # 💥 7-parameter spatio-temporal list
    params_list = [
        torch.tensor([np.log(init_phi1)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [0] log_phi1
        torch.tensor([np.log(init_phi2)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [1] log_phi2
        torch.tensor([np.log(init_phi3)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [2] log_phi3
        torch.tensor([np.log(init_phi4)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [3] log_phi4
        torch.tensor([init_advec_lat],         requires_grad=True, dtype=torch.float64, device=device_str ), # [4] advec_lat (NOT log)
        torch.tensor([init_advec_lon],         requires_grad=True, dtype=torch.float64, device=device_str ), # [5] advec_lon (NOT log)
        torch.tensor([np.log(init_nugget)],    requires_grad=True, dtype=torch.float64, device=device_str )  # [6] log_nugget
    ]

    # --- Define learning rates and parameter groups ---
    # 💥 Group all 7 parameters.
    # For a more advanced setup, you could create a separate group
    # for advection params [4, 5] with a different (e.g., smaller) LR.
    lr_all = lr 
    all_indices = [0, 1, 2, 3, 4, 5, 6] 
    
    param_groups = [
        {'params': [params_list[idx] for idx in all_indices], 'lr': lr_all, 'name': 'all_params'}
    ]

    # --- Print Job Info ---
    res_calc = (113 // lat_lon_resolution[0]) * (158 // lat_lon_resolution[0]) 
    print(f'\n--- Starting Day {day_idx+1} (2024-07-{day_idx+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n')
    for i, p in enumerate(params_list):
        print(f"  Param {i}: {p.item():.4f}")
            
    # --- 💥 Instantiate the Correct Class ---
    # Assumes fit_vecchia_adams is defined in your current scope
    model_instance = kernels_reparam_space_time.fit_vecchia_adams(
            smooth = v,
            input_map = daily_hourly_map,
            aggregated_data = daily_aggregated_tensor,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # --- Set Optimizer (This part is unchanged) ---
    optimizer, scheduler = model_instance.set_optimizer(
            param_groups,     
            lr=lr,            
            betas=(0.9, 0.99), 
            eps=1e-5, 
            scheduler_type='plateau', # Explicitly set to plateau
            patience=patience,        
            factor=factor             
        )

    # --- 💥 Call the Correct Fit Method ---
    out, epoch_ran = model_instance.fit_model(  # <-- Changed method name
            params_list,
            optimizer,
            scheduler, 
            model_instance.matern_cov_aniso_STABLE_log_reparam, # <-- Changed covariance function
            epochs=epochs
        )

    end_time = time.time()
    epoch_time = end_time - start_time
    
    print(f"Day {day_idx+1} optimization finished in {epoch_time:.2f}s over {epoch_ran+1} epochs.")
    print(f"Day {day_idx+1} final results (raw params + loss): {out}")


--- Starting Day 1 (2024-07-1) ---
Data size per day: 1092, smooth: 0.5
mm_cond_number: 8,
initial parameters: 

  Param 0: 3.0647
  Param 1: 0.3567
  Param 2: 0.1177
  Param 3: -4.6052
  Param 4: 0.0200
  Param 5: -0.0800
  Param 6: 0.4055


/opt/anaconda3/envs/faiss_env/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


--- Epoch 1 / Loss: 1.753064 ---
  Param 0: Value=3.0647, Grad=-0.23896844967440986
  Param 1: Value=0.3567, Grad=0.009605592633231486
  Param 2: Value=0.1177, Grad=-0.01976948523425638
  Param 3: Value=-4.6052, Grad=-0.07504262649484936
  Param 4: Value=0.0200, Grad=-0.32693794149868843
  Param 5: Value=-0.0800, Grad=1.1559085418106856
  Param 6: Value=0.4055, Grad=-0.17389288113657042
  Max Abs Grad: 1.155909e+00
------------------------------
--- Epoch 11 / Loss: 1.627234 ---
  Param 0: Value=3.6419, Grad=0.07594998746544755
  Param 1: Value=0.5097, Grad=-0.014314080436500561
  Param 2: Value=0.7000, Grad=0.015354697007560519
  Param 3: Value=-3.8439, Grad=-0.0022427608301070195
  Param 4: Value=-0.0555, Grad=-0.1015536503669124
  Param 5: Value=-0.3215, Grad=-0.09193484371903574
  Param 6: Value=1.0768, Grad=0.026480414662068595
  Max Abs Grad: 1.015537e-01
------------------------------
--- Epoch 21 / Loss: 1.602822 ---
  Param 0: Value=3.4599, Grad=0.009474985191501072
  Param 1:

# L bfgs

In [13]:
import torch
import numpy as np
import torch.optim as optim
import time
# NOTE: Assuming the necessary classes and global variables are defined elsewhere, 
# specifically the referenced 'fit_vecchia_lbfgs' class and 'kernels_reparam_space_time' module.

# --- Assume global variables are set: ---
# daily_hourly_maps, daily_aggregated_tensors, nns_map
# lat_lon_resolution, v, mm_cond_number, nheads
# lr, patience, factor, epochs

# --- L-BFGS SPECIFIC GLOBAL PARAMETERS ---
LBFGS_LR = 1.0
LBFGS_MAX_STEPS = 50       # Number of outer optimization steps
LBFGS_HISTORY_SIZE = 100   # Memory for Hessian approximation
LBFGS_MAX_EVAL = 50        # Max evaluations (line search) per step

# --- 2. Run optimization loop over pre-loaded data ---
day_indices = [0] 
for day_idx in day_indices:  

    # Assuming data access is correct
    daily_hourly_map = daily_hourly_maps[day_idx]
    daily_aggregated_tensor = daily_aggregated_tensors[day_idx]

    # --- Parameter Initialization (SPATIO-TEMPORAL) ---
    init_sigmasq   = 15.0
    init_range_lat = 0.66 
    init_range_lon = 0.7 
    init_nugget    = 1.5
    init_beta      = 0.1
    init_advec_lat = 0.02
    init_advec_lon = -0.08
    
    # Map model parameters to the 'phi' reparameterization
    init_phi2 = 1.0 / init_range_lon                # 1/range_lon
    init_phi1 = init_sigmasq * init_phi2            # sigmasq / range_lon
    init_phi3 = (init_range_lon / init_range_lat)**2  # (range_lon / range_lat)^2
    init_phi4 = init_beta**2                        # beta^2
    
    device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

    # 7-parameter spatio-temporal list (Log/Linear)
    params_list = [
        torch.tensor([np.log(init_phi1)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [0] log(phi1)
        torch.tensor([np.log(init_phi2)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [1] log(phi2)
        torch.tensor([np.log(init_phi3)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [2] log(phi3)
        torch.tensor([np.log(init_phi4)],      requires_grad=True, dtype=torch.float64, device=device_str ), # [3] log(phi4)
        torch.tensor([init_advec_lat],         requires_grad=True, dtype=torch.float64, device=device_str ), # [4] advec_lat (linear)
        torch.tensor([init_advec_lon],         requires_grad=True, dtype=torch.float64, device=device_str ), # [5] advec_lon (linear)
        torch.tensor([np.log(init_nugget)],    requires_grad=True, dtype=torch.float64, device=device_str )  # [6] log(nugget)
    ]

    # --- Define parameter groups ---
    lr_all = LBFGS_LR
    all_indices = [0, 1, 2, 3, 4, 5, 6] 
    
    # L-BFGS requires the parameters to be iterable in a single list or group
    param_groups = [
        {'params': [params_list[idx] for idx in all_indices], 'lr': lr_all, 'name': 'all_params'}
    ]

    # --- Print Job Info (using placeholder print variables) ---
    res_calc = (113 // lat_lon_resolution[0]) * (158 // lat_lon_resolution[0]) 
    print(f'\n--- Starting Day {day_idx+1} (2024-07-{day_idx+1}) ---')
    print(f'Data size per day: { res_calc }, smooth: {v}')
    print(f'mm_cond_number: {mm_cond_number},\ninitial parameters: \n')
    for i, p in enumerate(params_list):
        print(f"  Param {i}: {p.item():.4f}")
            
    # --- 💥 Instantiate the L-BFGS Class ---
    # NOTE: Assuming fit_vecchia_lbfgs is available via kernels_reparam_space_time
    model_instance = kernels_reparam_space_time.fit_vecchia_lbfgs(
            smooth = v,
            input_map = daily_hourly_map,
            aggregated_data = daily_aggregated_tensor,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    start_time = time.time()
    
    # --- 💥 Set L-BFGS Optimizer ---
    # L-BFGS specific arguments are passed here
    optimizer = model_instance.set_optimizer(
            param_groups,     
            lr=LBFGS_LR,            
            max_iter=LBFGS_MAX_EVAL,        # max_iter in LBFGS is the line search limit
            history_size=LBFGS_HISTORY_SIZE 
        )

    # --- 💥 Call the L-BFGS Fit Method ---
    out, steps_ran = model_instance.fit_vecc_lbfgs(
            params_list,
            optimizer,
            model_instance.matern_cov_aniso_STABLE_log_reparam, 
            max_steps=LBFGS_MAX_STEPS # Outer loop steps
        )

    end_time = time.time()
    epoch_time = end_time - start_time
    
    print(f"Day {day_idx+1} optimization finished in {epoch_time:.2f}s over {steps_ran+1} L-BFGS steps.")
    print(f"Day {day_idx+1} final results (raw params + loss): {out}")


--- Starting Day 1 (2024-07-1) ---
Data size per day: 1092, smooth: 0.5
mm_cond_number: 8,
initial parameters: 

  Param 0: 3.0647
  Param 1: 0.3567
  Param 2: 0.1177
  Param 3: -4.6052
  Param 4: 0.0200
  Param 5: -0.0800
  Param 6: 0.4055
--- Starting L-BFGS Optimization ---
--- Step 1/50 / Loss: 1.753064 ---
  Param 0: Value=3.1112, Grad=1.6542075974134585e-06
  Param 1: Value=0.5747, Grad=-7.918325239806735e-07
  Param 2: Value=0.6239, Grad=-1.0859535723617813e-06
  Param 3: Value=-3.1758, Grad=4.875418142862508e-07
  Param 4: Value=0.0386, Grad=3.5165812885850205e-06
  Param 5: Value=-0.2145, Grad=-8.485080755571978e-06
  Param 6: Value=1.2493, Grad=2.4363685241185446e-09
  Max Abs Grad: 8.485081e-06
------------------------------

Converged on gradient norm (max|grad| < 1e-05) at step 1
FINAL STATE: Step 1, Loss: 1.7530641938262586
  Raw (vecc) Parameters: [3.1112141446960853, 0.5747205647289292, 0.6238819518366033, -3.175843531960354, 0.03864259329907545, -0.21449336370243605, 